# Neural Networks with PyTorch

In this assignment, we are going to train a Neural Networks on the Japanese MNIST dataset. It is composed of 70000 images of handwritten Hiragana characters. The target variables has 10 different classes.

Each image is of dimension 28 by 28. But we will flatten them to form a dataset composed of vectors of dimension (784, 1). The training process will be similar as for a structured dataset.

<img src='https://drive.google.com/uc?id=16TqEl9ESfXYbUpVafXD6h5UpJYGKfMxE' width="500" height="200">

Your goal is to run at least 3 experiments and get a model that can achieve 80% accuracy with not much overfitting on this dataset.

Some of the code have already been defined for you. You need only to add your code in the sections specified (marked with **TODO**). Some assert statements have been added to verify the expected outputs are correct. If it does throw an error, this means your implementation is behaving as expected.

Note: You can only use fully-connected and dropout layers for this assignment. You can not convolution layers for instance

# 1. Import Required Packages

[1.1] We are going to use numpy, matplotlib and google.colab packages

In [ ]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt

# 2. Download Dataset

We will store the dataset into your personal Google Drive.


[2.1] Mount Google Drive

In [ ]:
drive.mount('/content/gdrive')

[2.2] Create a folder called `DL_ASG_1` on your Google Drive at the root level

In [ ]:
! mkdir -p /content/gdrive/MyDrive/DL_ASG_1

[2.3] Navigate to this folder

In [ ]:
%cd '/content/gdrive/MyDrive/DL_ASG_1'

[2.4] Show the list of item on the folder

In [ ]:
!ls

[2.4] Dowload the dataset files to your Google Drive if required

In [ ]:
import requests
from tqdm import tqdm
import os.path

def download_file(url):
    path = url.split('/')[-1]
    if os.path.isfile(path):
        print (f"{path} already exists")
    else:
      r = requests.get(url, stream=True)
      with open(path, 'wb') as f:
          total_length = int(r.headers.get('content-length'))
          print('Downloading {} - {:.1f} MB'.format(path, (total_length / 1024000)))
          for chunk in tqdm(r.iter_content(chunk_size=1024), total=int(total_length / 1024) + 1, unit="KB"):
              if chunk:
                  f.write(chunk)

url_list = [
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz'
]

for url in url_list:
    download_file(url)

[2.5] List the content of the folder and confirm files have been dowloaded properly

In [ ]:
! ls

# 3. Load Data

[3.1] Import the required modules from PyTorch

In [ ]:
# TODO (Students need to fill this section)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

[3.2] **TODO** Create 2 variables called `img_height` and `img_width` that will both take the value 28

In [ ]:
# TODO (Students need to fill this section)
img_height = 28
img_width = 28

[3.3] Create a function that loads a .npz file using numpy and return the content of the `arr_0` key

In [ ]:
def load(f):
    return np.load(f)['arr_0']

[3.4] **TODO** Load the 4 files saved on your Google Drive into their respective variables: x_train, y_train, x_test and y_test

In [ ]:
# TODO (Students need to fill this section)
x_train = load('kmnist-train-imgs.npz')
x_test = load('kmnist-test-imgs.npz')
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')


[3.5] **TODO** Using matplotlib display the first image from the train set and its target value

In [ ]:
# TODO (Students need to fill this section)
plt.imshow(x_train[0])
print(y_train[0])

# 4. Prepare Data

[4.1] **TODO** Reshape the images from the training and testing set to have the channel dimension last. The dimensions should be: (row_number, height, width, channel)

In [ ]:
# TODO (Students need to fill this section)
x_train = x_train.reshape(x_train.shape[0], img_height, img_width, 1)
x_test = x_test.reshape(x_test.shape[0], img_height, img_width, 1)

[4.2] **TODO** Cast `x_train` and `x_test` into `float32` decimals

In [ ]:
# TODO (Students need to fill this section)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

[4.3] **TODO** Standardise the images of the training and testing sets. Originally each image contains pixels with value ranging from 0 to 255. after standardisation, the new value range should be from 0 to 1.

In [ ]:
# TODO (Students need to fill this section)
x_train = x_train / 255
x_test = x_test / 255


[4.4] **TODO** Create a variable called `num_classes` that will take the value 10 which corresponds to the number of classes for the target variable

In [ ]:
# TODO (Students need to fill this section)
num_classes = 10

[4.5] **TODO** Convert the target variable for the training and testing sets to a binary class matrix of dimension (rows, num_classes).

For example:
- class 0 will become [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- class 1 will become [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
- class 5 will become [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
- class 9 will become [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
# TODO (Students need to fill this section)
y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)


# 5. Define Neural Networks Architecure

[5.1] Set the seed in PyTorch for reproducing results



In [ ]:
# TODO (Students need to fill this section)
torch.manual_seed(1)

[5.2] **TODO** Define the architecture of your Neural Networks and save it into a variable called `model`

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# TODO (Students need to fill this section)
layer_1 = nn.Linear(784, 512)
layer_2 = nn.Linear(512, 512)
layer_top = nn.Linear(512, 10)

In [ ]:
model = nn.Sequential(
    layer_1,
    nn.ReLU(),
    layer_2,
    nn.ReLU(),
    layer_top
)

[5.2] **TODO** Print the summary of your model

In [ ]:
# TODO (Students need to fill this section)
print(model)
model.to(device)

# 6. Train Neural Networks

[6.1] **TODO** Create 2 variables called `batch_size` and `epochs` that will  respectively take the values 128 and 500

In [ ]:
# TODO (Students need to fill this section)
batch_size = 128
epochs = 500

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
dataloader_train = DataLoader(x_train, batch_size = batch_size, shuffle = True)
dataloader_test = DataLoader(x_test, batch_size = batch_size, shuffle=True)

In [ ]:
len(dataloader_train)

[6.2] **TODO** Compile your model with the appropriate loss function, the optimiser of your choice and the accuracy metric

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Solution
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Assuming x_train, y_train, x_test, y_test are your data and target tensors
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

# Now, create the DataLoader objects
dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
data = data.view(data.size(0), -1)  # Flatten each image to a vector of size 784


In [ ]:
for i in range(5):
    total_loss = 0
    for data, target in dataloader_train:
        data = data.to(device)  # Move data to the same device as the model
        target = target.to(device)  # Move target to the same device as the model
        optimizer.zero_grad() #Zero gradients
        outputs = model(data) # Forward Propagation to get predicted outcome
        loss = criterion(outputs, target) # Compute the loss
        loss.backward()  # Back propagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    total_loss /= len(dataloader_train)
    print(f"EPOCH {i}: 'Loss' {total_loss:.4f}")

In [ ]:
# Solution
from sklearn.metrics import confusion_matrix
model.eval()
correct = 0
total = 0
# Get the predictions for the test dataset
predicted_labels = []
true_labels = []

with torch.no_grad():
    for data, target in dataloader_test:
        data = data.view(-1, input_size).to(device)
        target = target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(target.cpu().tolist())

In [ ]:
# TODO (Students need to fill this section)
model.

[6.3] **TODO** Train your model
using the number of epochs defined. Calculate the total loss and save it to a variable called total_loss.

In [ ]:
# TODO (Students need to fill this section)


[6.4] **TODO** Test your model.  Initiate the model.eval() along with torch.no_grad() to turn off the gradients.


In [ ]:
# TODO (Students need to fill this section)

# 7. Analyse Results

[7.1] **TODO** Display the performance of your model on the training and testing sets

In [ ]:
# TODO (Students need to fill this section)
print()

[7.2] **TODO** Plot the learning curve of your model

In [ ]:
# TODO (Students need to fill this section)
plt.

[7.3] **TODO** Display the confusion matrix on the testing set predictions

In [ ]:
# TODO (Students need to fill this section)
